In [ ]:
#Run this block only if the modules below are not installed
! pip install GPy
! pip install sklearn
! pip install scipy

In [ ]:
import os
import time
import itertools
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial import distance
from scipy.special import erfc
from scipy.stats import norm
import GPy

In [ ]:
#Parameter setting
BOA = 1   # 1 or 2
number_of_experiments_per_cycle = 5

#Label in DoE_Exp_Table_double_FoM.xlsx
output_label_main = "TON"
output_label_sub = "Chemoselectivity (%)"

#threshold of FoM 2
threshold_sub = 70.

In [ ]:
xi = 0.01
reject_rad = 1.
core_opt = False

if os.path.exists("./GP"):
    num = 1
    while True:
        if not os.path.exists(f"./GP_prev{num}"):
            os.rename("./GP", f"./GP_prev{num}")
            break
        else:
            num += 1
os.makedirs("./GP", exist_ok=True)

exp_table = pd.read_excel("./DoE_Exp_Table_double_FoM.xlsx", index_col=0)
exp_table.columns = [c.strip() for c in exp_table.columns]

x_data_column = [c for c in exp_table.columns if not (c==output_label_main or c==output_label_sub)]
print(f"[Data read] factors: {x_data_column}")

reso = 11
if len(x_data_column) > 6:
    print("[CAUTION] The number of the factors is large. The search grid resolution is lowered.")
    reso -= 2*(len(x_data_column)-6)
    print(f"Resolution={reso} (default:11)")
if reso < 5:
    print("[ERROR] The grid resolution is too low. Use sciCORE instead of this laptop.")
    print("System terminated.")
else:

    min_li = [exp_table.loc["MIN", c] for c in x_data_column]
    max_li = [exp_table.loc["MAX", c] for c in x_data_column]
    min_max_li = np.array([min_li, max_li], dtype=float)

    mmscaler = MinMaxScaler(feature_range=(0, 1), copy=True)
    mmscaler.fit(min_max_li)

    exp_table = exp_table.drop(["MIN", "MAX"])
    original_size = len(exp_table)
    start = time.time()
    
    if BOA == 1:
        for i in range(1, number_of_experiments_per_cycle+1):
            print(f"[Cycle {i}] {time.time()-start:.2f}[sec]")
            #print(exp_table)
            x_train = mmscaler.transform(exp_table.loc[:,x_data_column].values)
            y_main_train = exp_table.loc[:,[output_label_main]].values
            y_sub_train = exp_table.loc[:,[output_label_sub]].values
            #print(x_train); print(y_main_train); print(y_sub_train)

            kern_main = GPy.kern.RBF(len(x_data_column), ARD=True)
            gpy_model_main = GPy.models.GPRegression(X=x_train, Y=y_main_train, kernel=kern_main, normalizer=True)
            if core_opt: gpy_model_main.optimize(messages=True, max_iters=1e5)

            kern_sub = GPy.kern.RBF(len(x_data_column), ARD=True)
            gpy_model_sub = GPy.models.GPRegression(X=x_train, Y=y_sub_train, kernel=kern_sub, normalizer=True)
            if core_opt: gpy_model_sub.optimize(messages=True, max_iters=1e5)


            lis = []
            for j in range(len(x_data_column)):
                lis += [np.linspace(0, 1.0, reso)]
            points = np.array(list(itertools.product(*lis)))

            minDist = distance.cdist(points, x_train, metric='euclidean').min(axis=1)
            points = points[minDist>0.01]

            if i > 1:
                x_train_tentative = x_train[-(i-1):,:]
                #print(x_train_tentative)
                minDist = distance.cdist(points, x_train_tentative, metric='euclidean').min(axis=1)
                points = points[minDist>reject_rad]

            GO_table = pd.DataFrame(points, columns=[f"{c}_S" for c in x_data_column])

            #Calculation of main acquisition function
            pred_mean_main, pred_var_main = gpy_model_main.predict(points)
            pred_mean_main = pred_mean_main.reshape(-1)
            pred_std_main = np.sqrt(pred_var_main.reshape(-1))
            GO_table["pred_mean_main"] = pred_mean_main
            GO_table["pred_std_main"] = pred_std_main

            mu_sample, _ = gpy_model_main.predict(x_train)
            mu_sample_opt = np.max(mu_sample)

            with np.errstate(divide='warn'):
                imp = pred_mean_main - mu_sample_opt - xi
                Z = imp / pred_std_main
                ei = imp * norm.cdf(Z) + pred_std_main * norm.pdf(Z)
                ei[pred_std_main == 0.] = 0.

            GO_table["Acquisition_main"] = ei

            #Calculation of sub acquisition function
            pred_mean_sub, pred_var_sub = gpy_model_sub.predict(points)
            pred_mean_sub = pred_mean_sub.reshape(-1)
            pred_std_sub = np.sqrt(pred_var_sub.reshape(-1))
            GO_table["pred_mean_sub"] = pred_mean_sub
            GO_table["pred_std_sub"] = pred_std_sub

            #CFD of normal distribution (mu, std**2) from threshold_sub to infinity
            erfc_param = -1 * (threshold_sub - pred_mean_sub) / (np.sqrt(2) * pred_std_sub)
            GO_table["Acquisition_sub"] = 1 - (0.5 * erfc(erfc_param))

            #Calculation of total acquisition function
            GO_table["Acquisition_total"] = GO_table["Acquisition_main"]*GO_table["Acquisition_sub"]


            for c in x_data_column:
                GO_table[c] = 0.

            GO_table.loc[:,x_data_column] = mmscaler.inverse_transform(points)

            GO_table = GO_table.sort_values("Acquisition_total", ascending=False)
            GO_table[:1000].to_csv(f"./GP/GP_{i}.csv")

            next_index = len(exp_table)+1
            exp_table.loc[next_index] = -1
            top_data = GO_table.iloc[0]
            for clm in x_data_column:
                exp_table.loc[next_index, clm] = top_data[clm]
            exp_table.loc[next_index, output_label_main] = top_data["pred_mean_main"]
            exp_table.loc[next_index, output_label_sub] = top_data["pred_mean_sub"]
        
        
    elif BOA == 2:
        for i in range(1, number_of_experiments_per_cycle+1):
            print(f"[Cycle {i}] {time.time()-start:.2f}[sec]")
            x_train = mmscaler.transform(exp_table.loc[:,x_data_column].values)
            y_main_train_raw = exp_table.loc[:,[output_label_main]].values
            y_sub_train = exp_table.loc[:,[output_label_sub]].values

            #scaling y_main
            y_main_scale_val = y_main_train_raw.max()/10.
            y_main_train = y_main_train_raw/y_main_scale_val

            kern_main = GPy.kern.RBF(len(x_data_column), ARD=True)
            gpy_model_main = GPy.models.GPRegression(X=x_train, Y=y_main_train, kernel=kern_main, normalizer=True)
            if core_opt: gpy_model_main.optimize(messages=True, max_iters=1e5)

            kern_sub = GPy.kern.RBF(len(x_data_column), ARD=True)
            gpy_model_sub = GPy.models.GPRegression(X=x_train, Y=y_sub_train, kernel=kern_sub, normalizer=True)
            if core_opt: gpy_model_sub.optimize(messages=True, max_iters=1e5)

            lis = []
            for j in range(len(x_data_column)):
                lis += [np.linspace(0, 1.0, reso)]
            points = np.array(list(itertools.product(*lis)))

            minDist = distance.cdist(points, x_train, metric='euclidean').min(axis=1)
            points = points[minDist>0.01]

            GO_table = pd.DataFrame(points, columns=[f"{c}_S" for c in x_data_column])


            #Calculation of main acquisition function
            pred_mean_main, pred_var_main = gpy_model_main.predict(points)
            pred_mean_main = pred_mean_main.reshape(-1)
            pred_std_main = pred_var_main.reshape(-1)
            GO_table["pred_mean_main_real"] = pred_mean_main*y_main_scale_val
            GO_table["pred_mean_main_scaled"] = pred_mean_main
            GO_table["pred_std_main"] = pred_std_main

            mu_sample, _ = gpy_model_main.predict(x_train)
            mu_sample_opt = np.max(mu_sample)

            with np.errstate(divide='warn'):
                imp = pred_mean_main - mu_sample_opt - xi
                Z = imp / pred_std_main
                ei = imp * norm.cdf(Z) + pred_std_main * norm.pdf(Z)
                ei[pred_std_main == 0.] = 0.

            GO_table["Acquisition_main"] = ei

            #Calculation of sub acquisition function
            pred_mean_sub, pred_var_sub = gpy_model_sub.predict(points)
            pred_mean_sub = pred_mean_sub.reshape(-1)
            pred_std_sub = np.sqrt(pred_var_sub.reshape(-1))
            GO_table["pred_mean_sub"] = pred_mean_sub
            GO_table["pred_std_sub"] = pred_std_sub

            #CFD of normal distribution (mu, std**2) from threshold_sub to infinity
            erfc_param = -1 * (threshold_sub - pred_mean_sub) / (np.sqrt(2) * pred_std_sub)
            GO_table["Acquisition_sub"] = 1 - (0.5 * erfc(erfc_param))

            #Calculation of total acquisition function
            GO_table["Acquisition_total"] = GO_table["Acquisition_main"]*GO_table["Acquisition_sub"]

            for c in x_data_column:
                GO_table[c] = 0.

            GO_table.loc[:,x_data_column] = mmscaler.inverse_transform(points)

            GO_table = GO_table.sort_values("Acquisition_total", ascending=False)
            GO_table[:1000].to_csv(f"./GP/GP_{i}.csv")

            next_index = len(exp_table)+1
            exp_table.loc[next_index] = -1
            top_data = GO_table.iloc[0]
            for clm in x_data_column:
                exp_table.loc[next_index, clm] = top_data[clm]
            exp_table.loc[next_index, output_label_main] = top_data["pred_mean_main_real"]
            exp_table.loc[next_index, output_label_sub] = top_data["pred_mean_sub"]

    
    else:
        print("[ERROR] The setting 'BOA' must be 1 or 2.")
    
    for i in range(len(exp_table)+1):
        if i > original_size:
            exp_table.loc[i, output_label_main] = -1
            exp_table.loc[i, output_label_sub] = -1
    exp_table.to_excel("./GP/DoE_Result.xlsx")
    print(f"[Done] {time.time()-start:.2f}[sec]")
